<<9日K值連續三天<20後反彈，連續三天>80後反彈>>

設定要關注的股票  
if 庫存裡沒有此檔股票:  
    if 9日K值連續三天<20後反彈:  
        提醒使用者現在K值連續三天<20後反彈，建議買進  
elif 庫存裡有此檔股票:  
    if 現價-持有價格>停利標準:  
        提醒使用者現在已經達到預設的停利標準，建議脫手  
    elif 現價-持有價格>停損標準:  
        提醒使用者現在已經達到預設的停損標準，建議脫手  
    elif K<20:  
        提醒使用者現在K值連續三天<20後反彈，建議買進  
    elif K>80:  
        提醒使用者現在K值連續三天>80後反彈，建議賣出 

In [1]:
from bs4 import BeautifulSoup
# basic
import numpy as np
import pandas as pd
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 

#talib
import talib
from talib import abstract

#gui

import requests
import re
import io

import schedule
import time

In [2]:
from __future__ import print_function
import time
from linebot import (LineBotApi, WebhookHandler, exceptions)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *
import schedule
from pymongo import MongoClient
import urllib.parse
import datetime
import requests
from bs4 import BeautifulSoup

line_bot_api = LineBotApi('0RESBNWCfsvSRF4MMUJnu5uTKCeS3MFN05XAoDkNUmLh/JwfjZZqV1hisMl8GsR6wG175trlcY/74iN0sJ1A98hLE1v2takoD7UNfNY8Fu102jM7C6agGKWOQNAqYZzKK2sEh+eZtx86OO4bhxh6kAdB04t89/1O/w1cDnyilFU=')
yourid='Ud5ccf6452c79b7add21fcb8a008b0717'

In [3]:
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"

    response = requests.post(url)
    
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [4]:
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content)
    r = soup.find('string').text
    return r

In [5]:
def getK(stock):
    
    df = crawl_price(stock)
    start_time = datetime.datetime.now()-datetime.timedelta(days=365)
    end_time = datetime.date.today()
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
        'close':close.dropna().astype(float),
        'open':openp.dropna().astype(float),
        'high':high.dropna().astype(float),
        'low':low.dropna().astype(float),
        'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                    slowk_period=3,
                    slowk_matype=0,
                    slowd_period=3,
                    slowd_matype=0))

    data.reset_index(inplace=True)
    date = data.iloc[0:,0]
    K = data.iloc[0:,1]
    k=[]
    for i in range(len(K)):
        k.append(K[i])
    return k

In [6]:
def getPriceNow(stock):
    
    url2 = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryQuote5Price?compcode=" + stock
    client2 = get_webservice(url2)
    try:
        import xml.etree.cElementTree as ET
    except ImportError:
        import xml.etree.ElementTree as ET
    r2 = ET.fromstring(client2)
    stock_now = str(r2[0][0].text)#抓取現價
    return stock_now

In [7]:
def job(stock,sProfit,sLoss):
    #stock = "2330.tw"
    stock_now = getPriceNow(stock)
    stock_now = float(stock_now)
    k = getK(stock)
    url = "http://61.220.30.176/WebOrder/GVEAccount.asmx/QueryTodayPositionGve3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC&SubTotalItem=&SortItem=AssetCode%20Asc"
    client = get_webservice(url)

    try:
        import xml.etree.cElementTree as ET
    except ImportError:
        import xml.etree.ElementTree as ET

    r = ET.fromstring(client)
    stock_list=[]
    price=[]
    for country in r.findall('PortfolioAsset'):
            AssetCode = country.get('AssetCode')
            if AssetCode == stock:
                #print("找到",stock)
                stock_list.append(AssetCode)
                Cost = country.get('Cost')
                #print("價格",Cost)
                price.append(Cost)
                break
    if len(stock_list):
        teststock = stock_list[-1] 
        ownprice = float(price[-1])
        comprice = stock_now - ownprice
        if stock == teststock:
            if comprice >= float(sProfit):
                get = "持有股票已經達到預設的停利標準，建議脫手"
                print(get)
                #line_bot_api.push_message(yourid, TextSendMessage(text=get))
            elif comprice <= float(sLoss):
                get = "持有股票已經達到預設的停損標準，建議脫手"
                print(get)
                #line_bot_api.push_message(yourid, TextSendMessage(text=get))
            elif k[-2]<20 and k[-3]<20 and K[-4]<20:
                if k[-1]>20:
                    get = "K值已經連續三天小於20，今天反彈超越20，建議進場"
                    print(get)
                    #line_bot_api.push_message(yourid, TextSendMessage(text=get))
                else:
                    get = "尚未達到條件標準，不建議做交易"
                    print(get)
                    #line_bot_api.push_message(yourid, TextSendMessage(text=get))                
            elif k[-2]>80 and k[-3]>80 and k[-4]>80:
                if k[-1]<80:
                    get = "K值已經連續三天大於80，今天反彈小於80，建議將股票脫手"
                    print(get)
                    #line_bot_api.push_message(yourid, TextSendMessage(text=get))
                else:
                    get = "尚未達到條件標準，不建議做交易"
                    print(get)
                    #line_bot_api.push_message(yourid, TextSendMessage(text=get)) 
            else:
                get = "尚未達到條件標準，不建議做交易"
                print(get)
                #line_bot_api.push_message(yourid, TextSendMessage(text=get))
        
    else:
        if k[-2]<20 and k[-3]<20 and K[-4]<20:
            if k[-1]>20:
                get = "K值已經連續三天小於20，今天反彈超越20，建議進場"
                print(get)
                #line_bot_api.push_message(yourid, TextSendMessage(text=get))
            else:
                get = "尚未達到條件標準，不建議做交易"
                print(get)
                #line_bot_api.push_message(yourid, TextSendMessage(text=get))
        else:
            get = "尚未達到條件標準，不建議做交易"
            print(get)
            #line_bot_api.push_message(yourid, TextSendMessage(text=get))

In [8]:
stock = input("請輸入股票代碼:")
sProfit = input("請輸入停利標準:")
sLoss = input("請輸入停損標準:")
second_5_j = schedule.every(5).seconds.do(job,stock,sProfit,sLoss)
# 無窮迴圈
while True: 
    schedule.run_pending()
    time.sleep(1)

請輸入股票代碼:2330.tw
請輸入停利標準:5
請輸入停損標準:5
持有股票已經達到預設的停利標準，建議脫手
持有股票已經達到預設的停利標準，建議脫手
持有股票已經達到預設的停利標準，建議脫手
持有股票已經達到預設的停利標準，建議脫手
持有股票已經達到預設的停利標準，建議脫手
持有股票已經達到預設的停利標準，建議脫手


KeyboardInterrupt: 